In [2]:
!pip install -qU transformers torch sentencepiece accelerate>=0.20.1 rouge protobuf

In [3]:
from transformers import PegasusTokenizer
import os

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-cnn_dailymail")

In [ ]:
print(tokenizer.max_len_single_sentence)

1023


In [ ]:
text="Personally, I thought today was fine. Talking about APIs again really solidified my understanding of what they were. As for our group project, I don't feel great about it. Some of my team members have been collecting data for about two days straight. I can understand that some of our data sets can be harder to collect than others, but I feel like there's not tons of effort being put in by some of my team members. Today we decided to implement a new kind of data set to improve our model (recycling symbols on objects) and from what I can see on our Roboflow, only one team member collected and labeled his recycling symbol data. I don't know if my other team members just don't understand the idea, or if they are just not doing it. I'm planning on working on all of those other recycling labels tonight, and hopefully a data health check in our Roboflow to see how balanced our data sets are. I just kinda wish more of my team members were interactive with ideas and input."
tokens = tokenizer(text, truncation=True, padding="longest", max_length=1020, return_tensors="pt")

In [ ]:
tokens

{'input_ids': tensor([[17447,   108,   125,   666,   380,   140,  1226,   107, 20234,   160,
         17995,   435,   288, 47405,   161,  1301,   113,   180,   157,   195,
           107,   398,   118,   150,   456,   405,   108,   125,   272,   131,
           144,   393,   255,   160,   126,   107,  1027,   113,   161,   320,
           549,   133,   174,  6073,   335,   118,   160,   228,   390,  1870,
           107,   125,   137,   630,   120,   181,   113,   150,   335,  2120,
           137,   129,  4236,   112,  2690,   197,   536,   108,   155,   125,
           393,   172,   186,   131,   116,   146,  4692,   113,  1441,   270,
           414,   115,   141,   181,   113,   161,   320,   549,   107,  1984,
           145,  1159,   112,  3395,   114,   177,   660,   113,   335,   323,
           112,   927,   150,   861,   143,   216, 40638,  7205,   124,  3195,
           158,   111,   135,   180,   125,   137,   236,   124,   150, 38537,
         11347,   108,   209,   156,  

In [ ]:
len(tokens['input_ids'][0])

212

In [ ]:
folder_path = '/content/drive/MyDrive/NewsNebula/outputs'
for file_name in os.listdir(folder_path):
  if file_name == "file_46.txt":
    file_path = os.path.join(folder_path, file_name)
    print(file_path)
    with open(file_path, "r", encoding="utf-8") as file:
      text = file.read()
      print(text)
      tokenized_text = tokenizer(text, truncation=True, padding='max_length', max_length=1024, return_tensors="pt")
      print(tokenized_text['input_ids'][0])
      print(tokenized_text['attention_mask'][0])

/content/drive/MyDrive/NewsNebula/outputs/file_46.txt
trends reportera woman being carjacked while carrying her baby on wednesday received help from a local hero.the woman was taking her child out of her car outside a chick-fil-a in fort walton beach, florida, when a man approached her with a stick, according to the washington post. he then snatched her keys away and entered the vehicle as she screamed out for help.those cries were answered by chick-fil-a worker mikel gordon, who witnessed the ordeal and rushed to her aid. footage shows gordon getting into an altercation with the man, whom the okaloosa county sheriff’s office identified in a facebook post as 43-year-old william branch.“the victim was getting the infant out of her car at a restaurant at 743 beal parkway when she says branch approached her wielding a stick and demanding her keys,” the sheriff’s office wrote. “he then grabbed the keys from the waistband of her pants, opened the door, and got inside.”the one-minute video w

In [6]:
#To-Do: Create two list input_ids and attention_masks, store tokens for all the files stored in google drive
#1. Mount the google drive again
#2. Use os library - to extract text data from files
#3. Use tokenizer for all files
input_ids = []
attention_masks = []

folder_path = '/content/drive/MyDrive/NewsNebula/outputs'

for file_name in os.listdir(folder_path):
  if file_name.endswith(".txt"):
    file_path = os.path.join(folder_path, file_name)
    with open(file_path, "r", encoding="utf-8") as file:
      text = file.read()
      tokens = tokenizer(text, truncation=True, padding="max_length", max_length=500, return_tensors="pt")
      input_ids.append(tokens["input_ids"][0])
      attention_masks.append(tokens["attention_mask"][0])

In [7]:
from torch.utils.data import Dataset
class NewsDataset(Dataset):
  def __init__(self, input_ids, attention_masks):
    super().__init__()
    self.input_ids = input_ids
    self.attention_masks = attention_masks
  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self, index):
    return self.input_ids[index], self.attention_masks[index]

In [8]:
news_dataset = NewsDataset(input_ids, attention_masks)

In [9]:
import torch
path = '/content/drive/MyDrive/NewsNebula/my_dataset.pt'
torch.save(news_dataset, path)